# Python: Объекты занимают в памяти больше места, чем нужно

### Заур Шибзухов

In [1]:
import sys
from pprint import pprint
import collections
import recordclass

## В чем собственно проблема

* Python
* Ограниченная память
* Большое количество объектов одновременно живущих в памяти

> Как уменьшить объем памяти, занимаемой объектами?

## Dict

In [2]:
ob = {'x':1, 'y':2, 'z':3}
a = ob['x']

С версии 3.6: `Compact Dict` (введен под влиянием `PyPy`)

## Размер dict

In [3]:
print('sizeof:', sys.getsizeof(ob))

sizeof: 232


### Довольно большой объем памяти

* 1 000 000 экземпляров &rarr; 232 Мб
* 10 000 000 экземпляров &rarr; 2320 Мб

## Регулярный класс

In [4]:
class Point:
    #
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
ob = Point(1,2,3)
a = ob.x

In [5]:
print('Размер:', sys.getsizeof(ob), sys.getsizeof(ob.__dict__))

Размер: 48 104


> Словарь экземпляра (\_\_dict__) имеет меньший размер по сравнению с обычным словарем. <br/>
PEP 412: Key-Sharing Dictionary, Python 3.3+

## След экземпляра в памяти

`------------------------` <br/>
`PyGC_Head        16 байт` <br/>
`------------------------` <br/>
`PyObject_HEAD    16 байт` <br/>
`__weakref__      8  байт` <br/>
`__dict__         8  байт` <br/>
`------------------------` <br/>
**`ВСЕГО:           48 байт`**


### Тем не менее след в памяти все еще велик

* 1 000 000 экземпляров &rarr; 152 Мб
* 10 000 000 экземпляров &rarr; 1520 Мб 

## Регулярный класс + `__slots__`

In [6]:
class Point:
    __slots__ = 'x', 'y', 'z'
        
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

In [7]:
ob = Point(1,2,3)
print('Размер:', sys.getsizeof(ob))
print('Есть __dict__?', hasattr(ob, '__dict__'))
print('Есть __weakref__?', hasattr(ob, '__weakref__'))

Размер: 56
Есть __dict__? False
Есть __weakref__? False


## Структура экземпляра

`------------------------` <br/>
`PyGC_Head        16 байт` <br/>
`------------------------` <br/>
`PyObject_HEAD    16 байт` <br/>
`x                8  байт` <br/>
`y                8  байт` <br/>
`z                8  байт` <br/>
`------------------------` <br/>
**`ВСЕГО:           56 байт`**


### Компактный след в памяти

* 1 000 000 экземпляров &rarr; 56 Мб
* 10 000 000 экземпляров &rarr; 560 Мб

## За кулисами

In [8]:
pprint(Point.__dict__)

mappingproxy({'__doc__': None,
              '__init__': <function Point.__init__ at 0x7f452d08dca0>,
              '__module__': '__main__',
              '__slots__': ('x', 'y', 'z'),
              'x': <member 'x' of 'Point' objects>,
              'y': <member 'y' of 'Point' objects>,
              'z': <member 'z' of 'Point' objects>})


`x`, `y`, `z` в `Point.__dict__` специальные дескрипторы

## Tuple
> По существу, это запись, но без имен полей. 

При этом:

* доступ только по индексам. 

* доступ только для чтения.

In [9]:
ob = (1,2,3)
x, y, z = ob[0], ob[1], ob[2]

In [10]:
print("Размер:", sys.getsizeof(ob))

Размер: 64


## Структура экземпляра

`------------------------` <br/>
`PyGC_Head        16 байт` <br/>
`------------------------` <br/>
`PyObject_HEAD    16 байт` <br/>
`ob_size          8  байт` <br/>
`[0]              8  байт` <br/>
`[1]              8  байт` <br/>
`[2]              8  байт` <br/>
`------------------------` <br/>
**`ВСЕГО:           64 байт`**

### Компактный след в памяти

#### Но менее компактный, чем для регулярного класса со \_\_slots__

* 1 000 000 экземпляров &rarr; 64 Мб
* 10 000 000 экземпляров &rarr; 640 Мб

## Named Tuples

> Подкласс tuple с дескрипторами для доступа к элементам по имени

In [11]:
Point = collections.namedtuple("Point", "x y z")
ob = Point(1,2,3)
x = ob.x
y = ob[1]

## Структура экземпляра

`------------------------` <br/>
`PyGC_Head        16 байт` <br/>
`------------------------` <br/>
`PyObject_HEAD    16 байт` <br/>
`ob_size          8  байт` <br/>
`x                8  байт` <br/>
`y                8  байт` <br/>
`z                8  байт` <br/>
`------------------------` <br/>
**`ВСЕГО:           64 байт`**


## За кулисами

In [12]:
pprint(Point.__dict__)

mappingproxy({'__doc__': 'Point(x, y, z)',
              '__getnewargs__': <function Point.__getnewargs__ at 0x7f452c8701f0>,
              '__module__': '__main__',
              '__new__': <staticmethod object at 0x7f452c873040>,
              '__repr__': <function Point.__repr__ at 0x7f452c8700d0>,
              '__slots__': (),
              '_asdict': <function Point._asdict at 0x7f452c870160>,
              '_field_defaults': {},
              '_fields': ('x', 'y', 'z'),
              '_fields_defaults': {},
              '_make': <classmethod object at 0x7f452c8730d0>,
              '_replace': <function Point._replace at 0x7f452c85cc10>,
              'x': <_collections._tuplegetter object at 0x7f452c8730a0>,
              'y': <_collections._tuplegetter object at 0x7f452c873070>,
              'z': <_collections._tuplegetter object at 0x7f452c873130>})


## Mutable Tuple

> Мутируемый вариант namedtuple

* Естественный путь это реализовать его по примеру `namedtuple` на базе мутируемого вариант  `tuple`.
* Однако в Python нет встроенного мутируемого варианта `tuple`.
* В библиотеке **recordclass** создан тип `mutabletuple` &ndash; мутируемый вариант `tuple`.
  * `mutabletuple`  идентичен по структуре `tuple`
  * не имеет `PyGC_Head`.


In [13]:
mt = recordclass.mutabletuple(1,2,3)
print(mt)
print(mt[0], mt[1:])
print('Размеры:', 'mutabletuple:', sys.getsizeof(mt), 'tuple:', sys.getsizeof((1,2,3)))

mutabletuple(1, 2, 3)
1 mutabletuple(2, 3)
Размеры: mutabletuple: 48 tuple: 64


## След экземпляра в памяти

`------------------------` <br/>
`PyObject_HEAD    16 байт` <br/>
`ob_size          8  байт` <br/>
`[0]              8  байт` <br/>
`[1]              8  байт` <br/>
`[2]              8  байт` <br/>
`------------------------` <br/>
**`ВСЕГО:           40 байт`**


## Recordclass: мутируемый аналог namedtuple 

* Фабричная функция recordclass генерирует подкласс `mutabletuple` со специальными дескрипторами для доступа к полям по имени.
* `recordclass` API идентичен `namedtuple` API.
* Экземпляры классов, сгенерированных при помощи функции `recordclass` имеют след в памяти, меньший чем  сгенерированные при помощи `namedtuple` и экземпляры классов со `__slots__`.
  
  * Используется только **механизм подсчета ссылок**
  * **Циклическая сборка мусора** не поддерживается 

In [14]:
Point = recordclass.recordclass("Point", "x y z")
ob = Point(1,2,3)
print(ob)
print("Размер:", sys.getsizeof(ob))

Point(x=1, y=2, z=3)
Размер: 48


## След экземпляра в памяти

`------------------------` <br/>
`PyObject_HEAD    16 байт` <br/>
`ob_size          8  байт` <br/>
`x                8  байт` <br/>
`y                8  байт` <br/>
`z                8  байт` <br/>
`------------------------` <br/>
**`ВСЕГО:           48 байт`**


> След в памяти уменьшается на размер `PyGC_Head`.

## За кулисами

In [15]:
pprint(Point.__dict__)

mappingproxy({'__dict__': <property object at 0x7f452c872d60>,
              '__doc__': 'Point(x, y, z)',
              '__fields__': ('x', 'y', 'z'),
              '__getnewargs__': <function Point.__getnewargs__ at 0x7f452c870280>,
              '__getstate__': <function Point.__getstate__ at 0x7f452c87b040>,
              '__module__': '__main__',
              '__new__': <staticmethod object at 0x7f452c873910>,
              '__reduce__': <function Point.__reduce__ at 0x7f452c87b0d0>,
              '__repr__': <function Point.__repr__ at 0x7f452c8708b0>,
              '__slots__': (),
              '_asdict': <function Point._asdict at 0x7f452c870ca0>,
              '_make': <classmethod object at 0x7f452c873100>,
              '_replace': <function Point._replace at 0x7f452c870820>,
              'x': <recordclass.mutabletuple.mutabletuple_itemgetset object at 0x7f452d08ae30>,
              'y': <recordclass.mutabletuple.mutabletuple_itemgetset object at 0x7f452d08ae70>,
         

## Альтернативный форма

In [16]:
class Point(recordclass.RecordClass):
    x:int
    y:int
    z:int
        
ob = Point(1,2,3)
print(Point.__annotations__)
print(ob)
print("Размер:", sys.getsizeof(ob))

{'x': <class 'int'>, 'y': <class 'int'>, 'z': <class 'int'>}
Point(x=1, y=2, z=3)
Размер: 48


## Dataobject

In [17]:
class Point(recordclass.dataobject):
    x:int
    y:int
    z:int
        
ob = Point(1,2,3)
print(ob)
print("Размер:", sys.getsizeof(ob))

Point(x=1, y=2, z=3)
Размер: 40


## След экземпляра в памяти

`------------------------` <br/>
`PyObject_HEAD    16 байт` <br/>
`x                8  байт` <br/>
`y                8  байт` <br/>
`z                8  байт` <br/>
`------------------------` <br/>
**`ВСЕГО:           40 байт`**


## Альтернативный форма

In [18]:
Point = recordclass.make_dataclass("Point", {"x":int, "y":int, "z":int})
print(Point.__annotations__)
ob = Point(1,2,3)
print(ob)
print("Размер:", sys.getsizeof(ob))

{'x': <class 'int'>, 'y': <class 'int'>, 'z': <class 'int'>}
Point(x=1, y=2, z=3)
Размер: 40


## Dataarray

In [19]:
Point = recordclass.make_arrayclass("Point", 3)
ob = Point(1,2,3)
print(ob)
print("Размер:", sys.getsizeof(ob))

Point(1, 2, 3)
Размер: 40


## Классы с `Reference Counting` и без `Cyclic Garbage Collection`

> В начале Python поддерживал только механизм `Reference Counting`

> `Cyclic Garbage Collection` вместе с модулем `gc` был добавлен позже, как дополнительная возможность.

> Скоро `CGC` был распространен на все пользовательские классы, определяемые при помощи `class`

> CGC как универсальный механизм позволил разрешить основные проблемы, связанные с зацикливанием ссылок:

In [20]:
lst = [1,2,3]
lst.append(lst)
print(lst)

[1, 2, 3, [...]]


> CGC увеличил размер каждого экземпляра на величину `PyGC_Head`:

* 16 байт в Python 3.8
* 32-64 байт в Python < 3.8


## В каких случаях можно было бы отказаться от CGC?

#### Типичный пример это классы, представляющие структуры данных, в которых по контракту не предусмотрена возможность циклических ссылок

In [21]:
class Address:
    town: str
    street: str

class Person:
    name: str
    address: Address

> Возникновение циклических ссылок в таких случаях &ndash; результат ошибки

# Спасибо за внимание. 

# Вопросы?